# Combining & Exporting Data - Part 1

## Content
#### 1. Import libraries and data
#### 2. Data consistency check for df_ords_prior
#### 3. Combine df_ords and df_ords_prior
#### 4. Export df_merged_large

# 1. Import libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create project folder path
path = r'C:\Users\Lara\Career Foundry Projects\21-09-2023 Instacart Basket Analysis'

In [3]:
# Import datasets orders_products_prior.csv and orders_checked.csv
df_ords_prior = pd.read_csv (os.path.join(path, '02 Data', 'Original Data', 'orders_products_prior.csv'), index_col = False)
df_ords = pd.read_csv (os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), index_col = False)

In [4]:
# Check the output and shape for both df-s
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
df_ords_prior.shape

(32434489, 4)

In [6]:
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [7]:
df_ords.shape

(3421083, 6)

# 2. Data consistency check for df_ords_prior

In [9]:
# Check for mixed type columns
for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col)

#### No mixed data type columns found

In [10]:
# Find missing values in df_ords_prior
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

#### No missing values were found.

In [11]:
# Check for duplicates
# Create subset with full duplicates
df_ords_prior_dups = df_ords_prior[df_ords_prior.duplicated()]

In [12]:
# Check output
df_ords_prior_dups.head()

,order_id,product_id,add_to_cart_order,reordered


#### No duplicates were found.

# 3. Combine df_ords and df_ords_prior

In [13]:
# Merge df_ords and df_ords_prior with column 'order_id' as key
# Use default inner join and use merge flag
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [14]:
# Check output
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [15]:
# Check if it's full merge
df_merged_large['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [16]:
# Merge same 2 df-s with outer join to check if df_merged_large is indeed full merge
df_merged_large_out = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True, how = 'outer')

In [17]:
# Check output
df_merged_large_out.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,both
1,2539329,1,1,2,8,NaN,14084.0,2.0,0.0,both
2,2539329,1,1,2,8,NaN,12427.0,3.0,0.0,both
3,2539329,1,1,2,8,NaN,26088.0,4.0,0.0,both
4,2539329,1,1,2,8,NaN,26405.0,5.0,0.0,both


In [18]:
# Check if it's full merge
df_merged_large_out['_merge'].value_counts()

_merge
both          32434489
left_only       206209
right_only           0
Name: count, dtype: int64

In [19]:
# Create subset of rows with left_only flag
df_left = df_merged_large_out.loc[df_merged_large_out['_merge'] == 'left_only']

In [20]:
# Check output
df_left.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
59,1187899,1,11,4,8,14.0,NaN,NaN,NaN,left_only
255,1492625,2,15,1,11,30.0,NaN,NaN,NaN,left_only
344,2774568,3,13,5,15,11.0,NaN,NaN,NaN,left_only
363,329954,4,6,3,12,30.0,NaN,NaN,NaN,left_only
401,2196797,5,5,0,11,6.0,NaN,NaN,NaN,left_only


#### Inner merge is not full, but number 206209 of rows with left_only flags is very little compared to 32 millions rows and we will procede with df_merged_large

In [21]:
# Check shape of df_merged_large
df_merged_large.shape

(32434489, 10)

# 4. Export df_merged_large

In [22]:
# Export df_merged_large as pikle format
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))